<a href="https://colab.research.google.com/github/AnilOsmanTur/Spatio-Temporal-Event-Prediction/blob/main/Prediction_with_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction wtih MLP model

In [1]:
! pip install pytorch-lightning --quiet
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import pytorch_lightning as pl
import numpy as np

pl.seed_everything(42) # for reproducablity
print('imports done')

     |████████████████████████████████| 808kB 8.7MB/s 
     |████████████████████████████████| 276kB 18.2MB/s 
     |████████████████████████████████| 645kB 31.0MB/s 
     |████████████████████████████████| 829kB 37.1MB/s 
     |████████████████████████████████| 112kB 48.7MB/s 
     |████████████████████████████████| 1.3MB 47.6MB/s 
     |████████████████████████████████| 143kB 48.9MB/s 
     |████████████████████████████████| 296kB 48.0MB/s 


Global seed set to 42


imports done


## Data Loading and Spliting

In [2]:
data = np.load('data.npy')
n_sample = data.shape[0]
flat_data = data.reshape(n_sample, -1)
print('data shape after flattening', flat_data.shape)
n_train = int(0.8 * n_sample )
n_test = n_sample - n_train
train_data = flat_data[:n_train]
test_data = flat_data[n_train:]
print('Training data split', train_data.shape)
print('Testing data split', test_data.shape)

data shape after flattening (4500, 9)
Training data split (3600, 9)
Testing data split (900, 9)


In [3]:
# data generator function to split and connect small sequences
def data_generator(data, seq_size=5):
  n_sample = data.shape[0]
  dataset = []
  labels = []
  for end in range(seq_size,n_sample):
    start = end - seq_size
    dataset.append(data[start:end].flatten())
    labels.append(data[end])
  return np.array(dataset).astype(np.float32), np.array(labels).astype(np.float32)
print('Done')

Done


In [4]:
train_set, train_labels = data_generator(train_data, seq_size=5)
test_set, test_labels = data_generator(test_data, seq_size=5)
print(train_set.shape, train_labels.shape)
print(test_set.shape, test_labels.shape)

(3595, 45) (3595, 9)
(895, 45) (895, 9)


## Model Creation

In [26]:
class SpatioTempPredictor(pl.LightningModule):
  def __init__(self, seen_size=5):
    super().__init__()
    self.in_size = seen_size * 9
    self.predictor = nn.Sequential(
      nn.Linear(self.in_size, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 16),
      nn.ReLU(),
      nn.Linear(16, 9),
      nn.Sigmoid()
      )
    self.euclidian_dist = nn.PairwiseDistance(p=2)

  def hamming_dist(self, a, b):
    return torch.sum((a!=b).float())

  def forward(self, x):
    prediction = self.predictor(x)
    return prediction

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

  def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self(x)
    loss = F.mse_loss(y_hat, y)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self(x)
    loss = F.mse_loss(y_hat, y)
    e_dist = self.euclidian_dist(y_hat, y).sum()
    h_dist = self.hamming_dist(y_hat, y)

    # Calling self.log will surface up scalars for you in TensorBoard
    self.log('val_loss', loss, prog_bar=True)
    self.log('val_dist', e_dist, prog_bar=True)
    self.log('val_hdist', h_dist, prog_bar=True)

    return loss

  def test_step(self, batch, batch_idx):
    # Here we just reuse the validation_step for testing
    return self.validation_step(batch, batch_idx)


# data
dataset = list(zip(train_set, train_labels))
n_train = int(len(dataset)*0.9)
n_val = len(dataset) - n_train
train_split, val_split = torch.utils.data.random_split(dataset, [n_train, n_val])


train_loader = DataLoader(train_split, batch_size=16)
val_loader = DataLoader(val_split, batch_size=16)
test_loader = DataLoader(list(zip(test_set, test_labels)), batch_size=16)

# model
model = SpatioTempPredictor(seen_size=5)
print('Done')

Done


In [27]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# training
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=3,
   verbose=False,
   mode='max'
)
trainer = pl.Trainer(gpus=1, max_epochs=15, progress_bar_refresh_rate=20)
trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | predictor      | Sequential       | 5.7 K 
1 | euclidian_dist | PairwiseDistance | 0     
----------------------------------------------------
5.7 K     Trainable params
0         Non-trainable params
5.7 K     Total params
0.023     Total estimated model params size (MB)


Global seed set to 42


In [28]:
trainer.test(model, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_dist': 2.3915038108825684,
 'val_hdist': 140.2216339111328,
 'val_loss': 0.022880252450704575}
--------------------------------------------------------------------------------


[{'val_dist': 2.3915038108825684,
  'val_hdist': 140.2216339111328,
  'val_loss': 0.022880252450704575}]

In [29]:
# evaluation
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_dist': 2.385828971862793,
 'val_hdist': 140.2044677734375,
 'val_loss': 0.02285573072731495}
--------------------------------------------------------------------------------


[{'val_dist': 2.385828971862793,
  'val_hdist': 140.2044677734375,
  'val_loss': 0.02285573072731495}]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/